# Stock Market Ontology

### Dependencies

In [12]:
import csv
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import math
import os
import time
import numpy
import tinvest
from tinvest import CandleResolution
from rdflib import URIRef
from rdflib import Graph, Literal
from rdflib.namespace import RDF


### Metrics

In [13]:
fundamental_metrics = ['Index', 'P/E', 'P/S', 'Dividend %', 'Payout', 'Beta']

all_metrics = [*['Name', 'Sector', 'Country'], *fundamental_metrics]

### Get stocks tickers from [Nasdaq](https://www.nasdaq.com/market-activity/stocks/screener) and save to CSV
Use saved tickets to get and parse data from resource

In [14]:
# Read tickers from Nasdaq CSV file
def get_stock_ticker_list(file_name, ticker_field_name):
    tickers = []
    with open(file_name, newline='') as file:
        reader = csv.DictReader(file, [ticker_field_name])
        for row in reader:
            ticker = row[ticker_field_name]
            tickers.append(ticker)
    return tickers[1:]


### Parsing data from [FINVIZ](https://finviz.com/quote.ashx)

Received values:

* Name
* Sector
* Country
* P/E
* P/S
* Dividend %
* Payout
* Beta

In [15]:
resourceUrl = 'https://finviz.com/quote.ashx'


def get_fundamental_metric(soup, metric=None):
    if metric is None:
        metric = fundamental_metrics
    # Search in table with fundamental metrics
    name_cell = soup.find(text=metric) # First search header cell
    value_cell = name_cell.find_next(class_='snapshot-td2') # Next search closest cell
    return value_cell.text


def get_name_sector_country(soup):
    table = soup.find(attrs={'data-testid': 'quote-data-content'})
    links = table.findAll(class_='tab-link')

    name = links[0].find('b').text
    sector = links[1].text
    country = links[3].text

    return [('Name', name), ('Sector', sector), ('Country', country)]


def get_fundamental_data(df):
    notFound = []
    for symbol in df.index:
        try:
            headers = {
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}
            response = requests.get(resourceUrl + '?t=' + symbol, headers=headers)
            soup = bs(markup=response.content, features="html.parser")

            for metric in fundamental_metrics:
                metricValue = get_fundamental_metric(soup, metric)
                df.loc[symbol, metric] = metricValue

            name_sector_country = get_name_sector_country(soup)
            for field in name_sector_country:
                df.loc[symbol, field[0]] = field[1]
        except Exception as e:
            notFound.append(symbol)
        print('Parsed ticker:', symbol)

    return df, notFound

### Get data from [Tinkoff API](https://tinkoffcreditsystems.github.io/invest-openapi/)

Get monthly candles for the last two years

Calculate growth for each month

The expected growth calculate as mean growth for each month

The risk is calculated using the standard deviation formula


In [16]:
token = os.environ['TINKOFF_INVEST_TOKEN']
client = tinvest.SyncClient(token)


def calculate_risk_and_expect_growth_in_month(ticker):
    resp = make_request(client.get_market_search_by_ticker, ticker)
    instruments = resp.payload.instruments

    if len(instruments) == 0:
        return None

    figi = instruments[0].figi

    candlesHistoryInMonth = make_request(client.get_market_candles,
                                         figi,
                                         '2019-01-01T00:00:00.00+00:00',
                                         '2021-04-30T23:59:59.00+00:00',
                                         CandleResolution.month)

    prices = []
    for candle in candlesHistoryInMonth.payload.candles:
        prices.append((candle.h + candle.l) / 2)

    growths = []
    for i in range(0, len(prices) - 1):
        growths.append(math.log(prices[i + 1] / prices[i]))

    expect_growth = numpy.mean(growths)
    risk = numpy.std(growths)

    return expect_growth * 100, risk * 100  # To percent


def make_request(request, *args):
    try:
        return request(*args)
    except tinvest.TooManyRequestsError as e:
        print('Too many request, wait 5 minute')
        time.sleep(60 * 5 + 5)  # 5 min 5 sec
        return request(tuple(args))

### Save Received data from [FINVIZ](https://finviz.com/quote.ashx) and [Tinkoff API](https://tinkoffcreditsystems.github.io/invest-openapi/) in result csv

After that, we can read data from CSV and fill ontology

In [ ]:
def get_stocks(file_name='data.csv', separator=','):
    stocks = []
    table = pd.read_csv(file_name, sep=separator)
    for row in table.iterrows():
        stock = {}
        for col in table.columns:
            stock[col] = row[1][col]
        stocks.append(stock)

    return stocks

def get_companies_sectors_countries(file_name='data.csv', separator=','):
    table = pd.read_csv(file_name, sep=separator, parse_dates=[
        'Name', 'Sector', 'Country'
    ])

    allCompanies = set()
    allSectors = set()
    allCountries = set()

    for company in table['Name']:
        allCompanies.add(company)

    for sector in table['Sector']:
        allSectors.add(sector)

    for country in table['Country']:
        allCountries.add(country)

    return list(allCompanies), list(allSectors), list(allCountries)

### Declare Classes, Object properties and Data properties



In [ ]:
URI = 'http://www.semanticweb.org/matvey/ontologies/2021/4/stock-market-ontology#'


def to_URIRef(param):
    result = URIRef(URI + param)
    return result


# Classes
Asset = to_URIRef('Asset')
Bond = to_URIRef('Bond')
Company = to_URIRef('Company')
Country = to_URIRef('Country')
Currency = to_URIRef('Currency')
Dividend = to_URIRef('Dividend')
Fond = to_URIRef('Fond')
Growth = to_URIRef('Growth')
Index = to_URIRef('Index')
Sector = to_URIRef('Sector')
Stock = to_URIRef('Stock')

# Object properties
hasCompany = to_URIRef('hasCompany')
hasCountry = to_URIRef('hasCountry')
hasCurrency = to_URIRef('hasCurrency')
includeInIndex = to_URIRef('includeInIndex')
includeInSector = to_URIRef('includeInSector')

# Data properties
hasAverageDividendYield = to_URIRef('hasAverageDividendYield')
hasPE = to_URIRef('hasPE')
hasPS = to_URIRef('hasPS')
hasPayoutRatio = to_URIRef('hasPayoutRatio')
hasExpectGrowthInMonth = to_URIRef('hasExpectGrowInMonth')
hasRisk = to_URIRef('hasRisk')
hasBeta = to_URIRef('hasBeta')



### Fill the ontology with data


In [ ]:
def default_data_parser(value):
    return value


def percent_data_parser(value):
    return value.replace('%', '')


def default_name_parser(name):
    return name.strip()


def to_fixed(number, digits=2):
    return f"{number:.{digits}f}"


dataPropertyMap = {
    'P/E': {
        'relation': hasPE,
        'data_parser': default_data_parser
    },
    'P/S': {
        'relation': hasPS,
        'data_parser': default_data_parser
    },
    'Dividend %': {
        'relation': hasAverageDividendYield,
        'data_parser': percent_data_parser
    },
    'Payout': {
        'relation': hasPayoutRatio,
        'data_parser': percent_data_parser
    },
    'Beta': {
        'relation': hasBeta,
        'data_parser': percent_data_parser
    },
    'Expect Growth in month': {
        'relation': hasExpectGrowthInMonth,
        'data_parser': to_fixed
    },
    'Risk': {
        'relation': hasRisk,
        'data_parser': to_fixed
    }
}


def add_indexes(graph: Graph):
    indexes = ['DJIA', 'S&P500']
    return add_class_individuals_to_graph(graph, Index, indexes)


def add_companies_sectors_countries(graph: Graph):
    companies, sectors, countries = get_companies_sectors_countries()

    companiesResult = add_class_individuals_to_graph(graph, Company, companies, lambda name: name.replace(',', '')
                                                     .replace('.', '')
                                                     .strip())
    sectorsResult = add_class_individuals_to_graph(graph, Sector, sectors)
    countriesResult = add_class_individuals_to_graph(graph, Country, countries)

    return companiesResult, sectorsResult, countriesResult


def add_stocks(graph: Graph, all_indexes_nodes, all_companies_nodes, all_sectors_nodes, all_countries_nodes):
    stocks = get_stocks()

    result = {}
    for stock in stocks:
        ticker = stock['Ticker']
        stock_node = add_class_individual_to_graph(graph, Stock, ticker)
        result[ticker] = stock_node

        # Set object properties
        company_node = all_companies_nodes[stock['Name']]
        sector_node = all_sectors_nodes[stock['Sector']]
        country_node = all_countries_nodes[stock['Country']]
        indexes_nodes = get_indexes_nodes_for_stock(stock['Index'], all_indexes_nodes)

        add_object_property(graph, stock_node, company_node, hasCompany)
        add_object_property(graph, stock_node, sector_node, includeInSector)
        add_object_property(graph, stock_node, country_node, hasCountry)
        for index_node in indexes_nodes:
            add_object_property(graph, stock_node, index_node, includeInIndex)

        # Set data properties
        for key in dataPropertyMap.keys():
            dataPropertyInfo = dataPropertyMap[key]
            value = stock[key]
            if value == '-':
                continue
            parsed_value = dataPropertyInfo['data_parser'](value)
            add_data_property(graph, stock_node, parsed_value, dataPropertyInfo['relation'])

    return result


def add_class_individuals_to_graph(graph: Graph, Class: URIRef, names: list[string], parser=default_name_parser):
    result = {}
    for name in names:
        result[name] = add_class_individual_to_graph(graph, Class, name, parser)

    return result


def add_class_individual_to_graph(graph: Graph, Class: URIRef, name: string, parser=default_name_parser):
    parsed_name = parser(name)
    individual = to_URIRef(parsed_name.replace(' ', '_'))
    triple = (individual, RDF.type, Class)
    graph.add(triple)

    return individual


def add_object_property(graph, domain, client, relation):
    triple = (domain, relation, client)
    graph.add(triple)


def add_data_property(graph, domain, value, relation):
    triple = (domain, relation, Literal(value, datatype='xsd:float'))
    graph.add(triple)


def get_indexes_nodes_for_stock(stock_indexes: string, all_indexes_nodes):
    if stock_indexes == '-':
        return []

    parsed_stock_indexes = stock_indexes.replace(' ', '')

    result = []
    for key in all_indexes_nodes.keys():
        if key in parsed_stock_indexes:
            result.append(all_indexes_nodes[key])

    return result